<a href="https://colab.research.google.com/github/Hongjieun123/deeplearning-project/blob/master/%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

사전을 이용한 감성분석

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# 형태소분석기 관련 설치
!apt-get update
!apt-get install g++ openjdk-8-jdk

!pip install JPype1==0.7.4
!pip install rhinoMorph

In [5]:
# 경로 변경
%cd /content/gdrive/My\ Drive/pytest/

/content/gdrive/My Drive/pytest


In [6]:
#읽기 함수 정의
def read_data(filename, encoding='cp949'):
  with open(filename,'r',encoding=encoding) as f:
    data = [line.split('\t') for line in f.read().splitlines()]
    data = data[1:]    #txt 파일의 헤더는 제외
  return data

def write_data(data, filename, encoding='cp949'):     #쓰기 함수 정의
  with open(filename, 'w', encoding=encoding) as f:     #f.write(data)
    f.write(data)

data = read_data('ratings_morphed.txt')
print(len(data))
print(len(data[0]))
print(data[0])

197447
3
['8132799', '디자인 배우 학생 외국 디자이너 일군 전통 통하 발전하 가 문화 산업 부럽 사실 우리나라 그 어렵 시절 끝 열정 지키 노라노 같 전통 있 같 사람 꿈 꾸 이루 나가 있 감사하', '1']


In [0]:
#감정사전 읽기
data_id = [line[0] for line in data] #데이터 id
data_text = [line[1] for line in data] #데이터 본문
data_senti = [line[2] for line in data] #데이터 라벨 (긍/부정)

positive = read_data('positive.txt')   #긍정 감정사전 읽기
negative = read_data('negative.txt')   #부정 감정사전 읽기

pos_found = []
neg_found = []

[감정사전은 이렇게 생겼다.]
가능\n
가능하\n
가락\n
가치\n
간단하\n
간단히\n
간직하\n
간호\n
간호하\n
감동

In [8]:
#감정단어 파악

def cntWordInLine(data, senti):   #데이터, 감정사전을 인자로 받는다. 
  senti_found = []
  for onedata in data:     #데이터 한 줄씩 읽기 
    oneline_word = onedata.split(' ') #한 줄의 데이터를 공백단위로 분리 , 리스트로 저장
    senti_temp = 0
    for sentiword in senti:
      if sentiword[0] in oneline_word:   #posword[0] 하여 리스트를 문자열로 추출
        senti_temp += 1       #현재의 감정단어와 일치하면 숫자 올려줌 (중복안됌)
    senti_found.append(senti_temp)
  return senti_found

data_senti_poscnt = cntWordInLine(data_text, positive)
data_senti_negcnt = cntWordInLine(data_text, negative)

print(data_senti_poscnt)      #발견된 긍정 단어의 숫자 파악(리스트로 나와)
print(data_senti_negcnt)      #발견된 부정 단어의 숫자 파악


[5, 1, 0, 0, 2, 1, 0, 0, 0, 1, 1, 1, 0, 1, 2, 0, 1, 0, 1, 0, 3, 0, 0, 0, 3, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 2, 2, 2, 0, 1, 1, 1, 0, 1, 0, 3, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 0, 0, 2, 1, 0, 0, 0, 2, 2, 2, 2, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 2, 0, 0, 1, 1, 1, 2, 4, 0, 1, 0, 0, 0, 1, 3, 1, 2, 0, 0, 1, 2, 1, 0, 2, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 2, 0, 0, 0, 3, 0, 0, 0, 1, 1, 0, 3, 1, 1, 1, 1, 3, 1, 0, 4, 4, 1, 1, 3, 0, 1, 1, 0, 1, 0, 3, 2, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 2, 2, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 2, 0, 1, 3, 0, 2, 0, 1, 1, 0, 3, 1, 3, 0, 0, 0, 0, 1, 0, 1, 2, 0, 0, 2, 1, 1, 2, 1, 0, 1, 0, 0, 2, 1, 0, 1, 0, 1, 0, 1, 2, 0, 2, 1, 1, 1, 0, 2, 1, 1, 0, 1, 0, 1, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 1, 1, 1, 4, 0, 1, 1, 0, 0, 0, 0, 3, 0, 0, 0, 0, 1, 1, 0, 0, 1, 2, 2, 2, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 2, 0, 1, 0, 2, 0, 0, 1, 1, 1, 1, 2, 0, 1, 1, 1, 4, 2, 2, 0, 2, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 

In [9]:
#감정점수 계산
#Pandas 데이터프레임으로 저장
import pandas as pd
newdata = pd.DataFrame({'id':data_id, 'text':data_text, 'original':data_senti, 'pos':data_senti_poscnt, 'neg':data_senti_negcnt})
senti_score = newdata['pos'] - newdata['neg']
newdata['senti_score'] = senti_score

newdata.loc[newdata.senti_score > 0, 'new'] = 1  #새로운 긍부정 기호
newdata.loc[newdata.senti_score <=0, 'new'] = 0  #새로운 긍부정 기호

#처음에 기록된 긍부정과 새로 계산된 긍부정이 같은지 여부를 matched 컬럼에 저장
#original 부분이 '1' 문자로 되어있으므로 숫자로 변환 후 비교
newdata.loc[pd.to_numeric(newdata.original) == newdata.new, 'matched'] = 'True'
newdata.loc[pd.to_numeric(newdata.original)!= newdata.new, 'matched'] ='False'
newdata.head()

,id,text,original,pos,neg,senti_score,new,matched
0,8132799,디자인 배우 학생 외국 디자이너 일군 전통 통하 발전하 가 문화 산업 부럽 사실 우...,1,5,1,4,1.0,True
1,4655635,폴리스스토리 시리즈 뉴 없 최고,1,1,0,1,1.0,True
2,9251303,와 연기 진짜 쩔 지루하 생각하 몰입하 보 그래 이런 진짜 영화,1,0,1,-1,0.0,False
3,10067386,안개 자욱하 밤하늘 뜨 초승달 같 영화,1,0,0,0,0.0,False
4,2190435,사랑 해보 사람 처음 끝 웃 있 영화,1,2,0,2,1.0,True


In [10]:
#원점수와 비교 및 저장
score = newdata.matched.str.count('True').sum() / (newdata.matched.str.count('True').sum() + newdata.matched.str.count('False').sum()) * 100 #true 개수 / true + false 개수 * 100 = 정확도
print(score)

newdata.to_csv('newfile.csv', sep=',', encoding='cp949', index=False) #csv 저장
newdata.to_csv('newfile.txt', sep='\t', encoding='cp949', index=False) #txt 저장

62.40459465071639


In [0]:
#시그모이드 점수 계산
import numpy as np
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

newdata['sigmoid'] = newdata.senti_score.apply(sigmoid)  #apply(함수) score각각 sigmoid 넣음 0~1사이 값으로 변환

In [12]:
#결과 확인
pd.set_option('display.max_columns', 10)    #보여지는 컬럼 수 조정
print(newdata.iloc[:5,:])

         id                                               text original  pos  \
0   8132799  디자인 배우 학생 외국 디자이너 일군 전통 통하 발전하 가 문화 산업 부럽 사실 우...        1    5   
1   4655635                                  폴리스스토리 시리즈 뉴 없 최고        1    1   
2   9251303                와 연기 진짜 쩔 지루하 생각하 몰입하 보 그래 이런 진짜 영화        1    0   
3  10067386                              안개 자욱하 밤하늘 뜨 초승달 같 영화        1    0   
4   2190435                               사랑 해보 사람 처음 끝 웃 있 영화        1    2   

   neg  senti_score  new matched   sigmoid  
0    1            4  1.0    True  0.982014  
1    0            1  1.0    True  0.731059  
2    1           -1  0.0   False  0.268941  
3    0            0  0.0   False  0.500000  
4    0            2  1.0    True  0.880797  


In [13]:
# 훈련데이터와 테스트데이터 분리(자동)
data_text = [line[1] for line in data]        #데이터 본문
data_senti = [line[2] for line in data]          #데이터 긍부정 부분

from sklearn.model_selection import train_test_split      
train_data_text, test_data_text, train_data_senti, test_data_senti = train_test_split(data_text, data_senti, stratify=data_senti)   #stratify는 label이 0:25%, 1:75%라면 train도 그 비율 유지

#Counter class 이용하여 각 분류가 훈련데이터와 테스트 데이터에 같은 비율로 들어 갔는지 확인
from collections import Counter
train_data_senti_freq = Counter(train_data_senti)
print('train_data_senti_freq: ', train_data_senti_freq)

test_data_senti_freq = Counter(test_data_senti)
print('test_data_senti_freq: ', test_data_senti_freq)

#train, test 는 75:25 비율로 나타난다. 
print(len(train_data_text))

train_data_senti_freq:  Counter({'1': 74098, '0': 73987})
test_data_senti_freq:  Counter({'1': 24699, '0': 24663})
148085


In [0]:
#data generation
class DataGeneration:

  def __init__(self, data, seperation_rate):

    self.data = data

    self.seperation_rate = seperation_rate

  def generate(self):
    print('data.shape = ', self.data)
    
    total_data_num = len(self.data)
    test_data_num = int(len(self.data) * self.seperation_rate)

    np.random.shuffle(self.data)

    test_data = self.data[ :test_data_num ]
    training_data = self.data[ test_data_num: ]

    return training_data , test_data

In [15]:
seperation_rate = 0.3

data_obj1 = DataGeneration(data_text, seperation_rate)
data_obj2 = DataGeneration(data_senti, seperation_rate)

(training_data, test_data) = data_obj1.generate()
(train_senti, test_senti) = data_obj2.generate()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
#행렬 형태로 변환
from sklearn.feature_extraction.text import CountVectorizer #dtm 만들려면 행렬로 변환하는게 편하다.
vect = CountVectorizer(min_df=5).fit(train_data_text) #빈도 5이상만
X_train = vect.transform(train_data_text)
print('X_train:\n', repr(X_train))
#전체중에 821624만 차있고, 나머지는 비어있다. min_df를 높여줄 필요가 있다. 

X_train:
 <148085x12379 sparse matrix of type '<class 'numpy.int64'>'
	with 821234 stored elements in Compressed Sparse Row format>


In [45]:
#행렬 내용 관찰
feature_names = vect.get_feature_names() #column 수
print('특성 개수:' ,len(feature_names))
print('처음 20개 특성:\n', feature_names[:20])
print('3000개 ~ 5000개 까지 특성:',feature_names[3000:5000])

특성 개수: 12319
처음 20개 특성:
 ['007', '10점', '1빠', '24', 'cgv', 'ebs', 'jtbc', 'kbs', 'la', 'mb', 'mbc', 'naver', 'new', 'ok', 'sbs', 'sns', 'tv', 'ㄴㄴ', 'ㄷㄷ', 'ㅂㄷㅂㄷ']
3000개 ~ 5000개 까지 특성: ['러브레터', '러셀', '러쉬', '러시아', '러시아어', '러시아워', '러시안', '럭키', '런닝', '런던', '런지', '레고', '레나', '레드', '레디', '레먼', '레미제라블', '레베카', '레벨', '레비', '레빗', '레슬러', '레슬리', '레슬링', '레아', '레알', '레오', '레옹', '레이', '레이놀즈', '레이드', '레이디', '레이스', '레이싱', '레이저', '레이첼', '레이코', '레인', '레저', '레전드', '레져', '레즈', '레즈비언', '레지던트', '레츠', '레파토리', '레포트', '렉스', '렉터', '렌즈', '로간', '로건', '로그', '로그인', '로더', '로드', '로드리게스', '로드리게즈', '로드무비', '로또', '로라', '로랑', '로렌스', '로로', '로리타', '로마', '로만', '로망', '로망스', '로맨스', '로맨틱', '로맨틱하', '로메', '로미오', '로버츠', '로버트', '로베르토', '로보', '로보캅', '로보트', '로봇', '로비', '로빈', '로빈스', '로스', '로스트', '로얄', '로완', '로우', '로이', '로인', '로저', '로즈', '로지', '로치', '로케', '로케이션', '로켓', '로코', '로키', '로튼', '로페즈', '록산', '록키', '롤러코스터', '롤링', '롯데', '롯데시네마', '롱테이크', '료코', '루가', '루나', '루니', '루머', '루비', '루스', '루시', '루이', '루이스', '루저', '루즈', '루카', '루카스', '루크', '루

In [17]:
#머신러닝 알고리즘 적용
import pandas as pd
from sklearn.model_selection import cross_val_score #cross_validation 교차검증
from sklearn.linear_model import LogisticRegression #선형회귀
y_train = pd.Series(train_data_senti)  #리스트 형태를 종속변수가 될 수 있는 1차원 배열로 만든다.

lr = LogisticRegression(solver='liblinear')    #샘플이 특정 클래스에 속할 확률 추정 
lr.fit(X_train, y_train)                    #훈련

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [19]:
#테스트 데이터 입력
X_test = vect.transform(test_data_text)   #test 값 넣어준다.
Y_test = pd.Series(test_data_senti)  #열 생성
print('테스트 데이터 점수:', lr.score(X_test, Y_test))

테스트 데이터 점수: 0.8112110530367489


In [20]:
#형태소분석기 시작
import rhinoMorph
rn = rhinoMorph.startRhino()    #형태소분석기 시동걸음

#형태소 분석
new_input = '오늘은 정말 재미있는 하루구나!'
input_data = []
morphed_input = rhinoMorph.onlyMorph_list(rn, new_input, pos=['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'])
morphed_input = ' '.join(morphed_input) #리스트로 변환되므로 ,없이 하나의 문장으로 표시해주기 위해

input_data.append(morphed_input)  #하나의 리스트로 출력해주기 위해
print('input data = ', input_data)



filepath:  /usr/local/lib/python3.6/dist-packages
classpath:  /usr/local/lib/python3.6/dist-packages/rhinoMorph/lib/rhino.jar
RHINO started!
input data =  ['오늘 정말 재미있 하루']


In [21]:
X_input = vect.transform(input_data)
result = lr.predict(X_input)

if result == '0':
  print('부정적인 글입니다')
else:
  print('긍정적인 글입니다')

긍정적인 글입니다
